In [ ]:
# Energy prediction 3 (ASHRAE)
# Imports/Formatting
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from tensorflow import keras


import warnings
warnings.filterwarnings('ignore')


In [ ]:
'''
README
With getting processed data, this programme reads in as feather, and then converts
it to CSV. Google colab RAM cannot handle reading them both in as CSV.
'''

'\nREADME\nWith getting processed data, this programme reads in as feather, and then converts\nit to CSV. Google colab RAM cannot handle reading them both in as CSV.\n'

In [ ]:
# Get PROCESSED train data from drive 
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
train_data = pd.read_feather('/content/drive/My Drive/training_data_processed_feather.feather')

Mounted at /content/drive


In [ ]:
# Get PROCESSED test data from drive
test_data = pd.read_feather('/content/drive/My Drive/testing_data_processed_feather.feather')

In [ ]:
# Convert training data back to CSV file
train_data.to_csv('train_data', index = False)

In [ ]:
# Convert testing data back to CSV file
test_data.to_csv('test_data', index = False)

In [ ]:
# Check type
print('Type of train_data: ', type(train_data))
print('Type of test_data: ', type(test_data))

Type of train_data:  <class 'pandas.core.frame.DataFrame'>
Type of test_data:  <class 'pandas.core.frame.DataFrame'>


In [ ]:
# Drop index columns
train_data = train_data.drop('index', axis = 1, errors = 'ignore')
test_data = test_data.drop('index', axis = 1, errors = 'ignore')

In [ ]:
# Inspect train_data
train_data.head()

,building_id,square_feet,primary_use,meter,air_temperature,day_of_year,hour,isDayTime,day_of_week,meter_reading
0,46,9.109375,11,0,25.000000,1.0,0.0,0,4.0,3.994141
1,74,12.867188,8,0,25.000000,1.0,0.0,0,4.0,3.785156
2,93,10.414062,6,0,25.000000,1.0,0.0,0,4.0,3.978516
3,105,10.835938,0,0,3.800781,1.0,0.0,0,4.0,3.191406
4,106,8.585938,0,0,3.800781,1.0,0.0,0,4.0,0.318115


In [ ]:
# Inspect test_data
test_data.head()

,building_id,square_feet,primary_use,meter,air_temperature,day_of_year,hour,isDayTime,day_of_week
0,0,8.914062,0,0,17.796875,1.0,0.0,0,6.0
1,1,7.910156,0,0,17.796875,1.0,0.0,0,6.0
2,2,8.593750,0,0,17.796875,1.0,0.0,0,6.0
3,3,10.070312,0,0,17.796875,1.0,0.0,0,6.0
4,4,11.664062,0,0,17.796875,1.0,0.0,0,6.0


In [ ]:
'''
--------------------------------------------------------------------------------
IMPLEMENTATION
'''

'\n--------------------------------------------------------------------------------\nIMPLEMENTATION\n'

In [ ]:
# Get X_train and y_train
X_train = train_data.drop('meter_reading', axis = 1)
y_train = train_data['meter_reading'].values

In [ ]:
 # Split data for test and train
train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
# Define RMSLE specifically for neural network
from keras import backend as k
def NN_RMSLE(y_act, y_pred):
  return k.sqrt(k.mean(k.square(y_pred - y_act)))

In [ ]:
# Split into train and validation sets
train_xx, val_xx, train_yy, val_yy = train_test_split(train_x, train_y, test_size = 0.2, random_state = 42)

In [ ]:
# Define model
from keras.models import Sequential
from keras import layers
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

model = Sequential()

# Add layers, etc
model.add(layers.Dense(512, activation = 'relu', input_shape = (train_xx.shape[1], )))
model.add(Dense(1, activation = 'linear'))
model.compile(optimizer = 'adam', loss = NN_RMSLE)

# Fit model
model.fit(train_xx, train_yy, epochs = 50, batch_size = 2048, validation_data = (val_xx, val_yy))

Epoch 1/50
5732/5732 [==============================] - 46s 8ms/step - loss: 1.5143 - val_loss: 1.3778
Epoch 2/50
5732/5732 [==============================] - 43s 7ms/step - loss: 1.3611 - val_loss: 1.2921
Epoch 3/50
5732/5732 [==============================] - 43s 7ms/step - loss: 1.2961 - val_loss: 1.3638
Epoch 4/50
5732/5732 [==============================] - 46s 8ms/step - loss: 1.2632 - val_loss: 1.2597
Epoch 5/50
5732/5732 [==============================] - 49s 9ms/step - loss: 1.2376 - val_loss: 1.1630
Epoch 6/50
5732/5732 [==============================] - 48s 8ms/step - loss: 1.2197 - val_loss: 1.2157
Epoch 7/50
5732/5732 [==============================] - 48s 8ms/step - loss: 1.2034 - val_loss: 1.1950
Epoch 8/50
5732/5732 [==============================] - 46s 8ms/step - loss: 1.1916 - val_loss: 1.2766
Epoch 9/50
5732/5732 [==============================] - 43s 7ms/step - loss: 1.1796 - val_loss: 1.1338
Epoch 10/50
5732/5732 [==============================] - 46s 8ms/step - l

In [ ]:
print('Neural Network Training RMSLE: {}'.format(model.evaluate(train_x, train_y, verbose = 0)))
print('Neural Network Testing RMSLE: {}'.format(model.evaluate(test_x, test_y, verbose = 0)))

Neural Network Training RMSLE: 1.0582715272903442
Neural Network Testing RMSLE: 1.0591145753860474


In [ ]:
# Use model to make a prediction 
prediction = model.predict(test_data)

In [1]:
# Show prediction
prediction

NameError: ignored

In [ ]:
# Inspect and flatten prediction
prediction_flat = prediction.flatten()
len(prediction_flat)

41697600

In [ ]:
# Create row_id column for Kaggle submission
row_id = (np.arange(0, len(prediction_flat))).astype(np.int32)

In [ ]:
# Verify row_id is the required ~ 41 million entries
len(row_id)

In [ ]:
#df = pd.DataFrame({'row_id':row_id, 'meter_reading':prediction_flat})

In [ ]:
#df.to_csv('/content/drive/My Drive/RNN_prediction.csv', index = False)

In [ ]:
#df.head()

In [ ]:
# Construct dataframe
df_2 = pd.DataFrame({'row_id': row_id, 'meter_reading':np.expm1(prediction_flat)})

In [ ]:
# Save as csv
df_2.to_csv('/content/drive/My Drive/RNN_prediction_expm1.csv', index = False)